In [7]:
from io import BytesIO
from IPython.display import clear_output, Image, display
from PIL import Image as Img
from glob import glob
import cv2
import numpy as np

In [8]:
def set_img(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (400,300))
    return img

def show_video(frame):
    img = Img.fromarray(frame, "RGB")
    buffer = BytesIO()
    img.save(buffer, format="JPEG")
    display(Image(data=buffer.getvalue()))
    clear_output(wait=True)

def draw_boxes(img, boxes, color=(0, 255, 0)):
    for box in boxes:
        cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), color, 2)
    return img

In [3]:
# Open webcam
webcam = cv2.VideoCapture(0)

try:
    while(True):
        video_ok, frame = webcam.read()

        if webcam:
            frame = set_img(frame)
            show_video(frame)

except KeyboardInterrupt:
    webcam.release()
    print("Break")

Break


In [11]:
# Coletando imagens para treinamento
webcam = cv2.VideoCapture(0)
class_face = cv2.CascadeClassifier('classificadores/haarcascade_frontalface_default.xml')
count = 0
webcam_name = 'greg'

try:
    while(True):
        video_ok, frame = webcam.read()

        if video_ok:
            frame = set_img(frame)
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
            faces = class_face.detectMultiScale(frame_gray, 1.3, 5)
            if len(faces) > 0:
                count += 1
                if count <= 100:
                    for (x,y,w,h) in faces:
                        roi = frame_gray[y:y+h, x:x+w]
                        cv2.resize(roi, (200, 200), interpolation=cv2.INTER_LANCZOS4)
                        cv2.imwrite(f"imagens/{webcam_name}/" + str(count) + ".png", roi)
                    cv2.putText(frame, "Coletado " + str(count) + " faces", (20,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 2)
                else:
                    cv2.putText(frame, "ConcluIdo", (20,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 2)

            show_video(frame)

except KeyboardInterrupt:
    webcam.release()
    print("Break")

Break


In [27]:
# Listando imagens coletadas
faces_train = f"imagens/{webcam_name}/"
list_faces = [img for img in glob(faces_train + "*.png")]

In [31]:
### Exibir imagens coletadas em grade
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 10))
for e, img in enumerate(list_faces):
    plt.subplot(1, 3, e + 1)
    plt.title(f"Img {img}")
    imagem = cv2.imread(img)
    plt.imshow(imagem)

plt.show()

In [14]:
# Treinando o modelo
data_train, subjects = [], []

for img in list_faces:
    imagem = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    data_train.append(imagem)
    subjects.append(0)

subjects = np.asarray(subjects, dtype=np.int32)
model_lbph = cv2.face.LBPHFaceRecognizer_create()
model_lbph.train(data_train, subjects)

In [15]:
# Reconhecendo faces
webcam = cv2.VideoCapture(0)
class_face = cv2.CascadeClassifier('classificadores/haarcascade_frontalface_default.xml')
count = 0

try:
    while(True):
        video_ok, frame = webcam.read()

        if video_ok:
            frame = set_img(frame)
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
            faces = class_face.detectMultiScale(frame_gray, 1.3, 5)

            if len(faces) > 0:
                for (x,y,w,h) in faces:
                    roi = frame_gray[y:y+h, x:x+w]
                    roi = cv2.resize(roi, (200, 200), interpolation=cv2.INTER_LANCZOS4)
                    predict = model_lbph.predict(roi)
                    cv2.putText(frame, "Similaridade " + str(predict[1]), (x,y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 2)
                    draw_boxes(frame, [(x,y,x+w,y+h)], (255,0,0))
            show_video(frame)

except KeyboardInterrupt:
    webcam.release()
    print("Break")

Break
